In [1]:
import json
import pandas as pd
import numpy as np
import os
import time
import requests

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getMedianRawStakeSnapshot():
    method = "hmy_getMedianRawStakeSnapshot"
    params = []
    return get_information(method, params)['result']

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [5]:
def M5_test():
    flag = True
    # get the median stake and the upper and lower level 
    result = getMedianRawStakeSnapshot()
    median = int(float(result['epos-median-stake']))
    lower = int(median- 0.15*median)
    upper = int(median + 0.15*median)
    print("median stake is " + str(median))
    print("lower bond is " + str(lower))
    print("upper bond is " + str(upper))
    
    validator_infos = result['epos-slot-winners']
    count = 0
    for i in validator_infos:
        addr = i['slot-owner']
        stake = int(float(i['eposed-stake']))
        bls_key = i['bls-public-key']
        count += 1
        if stake > upper or stake < lower:
            print("validator: ", addr, " bls public key: ", bls_key) 
            print("effective stake is out of range. The effective stake is ", stake)
            print("")
            flag = False

    print("total slots verified:", count)
    if flag:
        return True
    else:
        return False

In [6]:
M5_test()

median stake is 709048999999999945211904
lower bond is 602691649999999946719232
upper bond is 815406349999999876595712
validator:  one1hwn68w7m5g8daltvrjkmq23qn58nrvr3yz6ryt  bls public key:  48ad00d97fb657f3476b5a8466798f33a23b16615091ebcd3587c14c9aad9e2e28c28fd12a7e9c887e621eb2f7860d00
effective stake is out of range. The effective stake is  815406350000000010813440

validator:  one1syx0rmtttr8jehlxvnkpyx4hvrlevmcx3tcghy  bls public key:  02760cadc6ef3d3c1f8b66adbe182ebc252b451178efc168e856870311c8a7005c7060d28ea72c6c7adfab606989f413
effective stake is out of range. The effective stake is  815406350000000010813440

validator:  one19cuygr776f7j9ep97hjm0np9ay6nus9w5msy0n  bls public key:  be205ec86841a3b96249f80e364c416a63b938617ade36b6798b377188d99bf27b5e85189116f4a4ba1b7a0c060cb115
effective stake is out of range. The effective stake is  815406350000000010813440

validator:  one1jl3p3huw0yzsnsnpdux3vjsy9v97rnxgp2gptu  bls public key:  1d15096474cd0a46cea0d7b63d5b8a0e4b242348e96969110

False